<a href="https://colab.research.google.com/github/vedantzope9/ML_mspa/blob/main/MSPA_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [45]:
#For Labelled dataset
df=pd.read_csv("/content/drive/MyDrive/diabetes_dataset.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Linear Regression
df=df.dropna()
x = df.drop('Outcome', axis=1)
y = df['Outcome']

In [14]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [15]:

model = tf.keras.Sequential([
   tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1)
])

In [16]:
model.compile(optimizer='sgd', loss='mean_squared_error')


In [19]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [20]:
y_pred = model.predict(X_test)

5/5 [==============================] - 0s 3ms/step


In [21]:
mse=mean_squared_error(y_test,y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.16811802036551943


In [33]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [41]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [42]:
y_pred_knn = knn.predict(X_test)


In [43]:
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("k-NN Accuracy:", accuracy_knn)

k-NN Accuracy: 0.6948051948051948


In [92]:
#For Uabelled dataset
netflix_data=pd.read_csv("/content/drive/MyDrive/netflix_titles.csv")
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [93]:
#KMeans
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:

# Preprocess descriptions, fill missing values with empty strings
corpus = netflix_data['description'].fillna('')

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Convert the TF-IDF matrix to a dense NumPy array
tfidf_dense = tfidf_matrix.toarray()

# Define the number of clusters (e.g., 5 clusters)
n_clusters = 5

# Initialize cluster centroids by randomly selecting data points
initial_indices = np.random.choice(tfidf_dense.shape[0], n_clusters, replace=False)
initial_centroids = tf.Variable(tf.constant(tfidf_dense[initial_indices], dtype=tf.float32))

# K-Means clustering loop
for _ in range(100):
    distances = tf.reduce_sum(tf.square(tf.expand_dims(tf.constant(tfidf_dense, dtype=tf.float32), axis=1)) - tf.expand_dims(initial_centroids, axis=0), axis=2)

    # Assign data points to the nearest cluster
    cluster_assignments = tf.argmin(distances, axis=1)

    # Update cluster centroids
    new_centroids = tf.concat([
        tf.reduce_mean(tf.boolean_mask(tf.constant(tfidf_dense, dtype=tf.float32), tf.equal(cluster_assignments, i), axis=0)
        for i in range(n_clusters)
    ], axis=0)

    # Assign the updated centroids
    initial_centroids.assign(new_centroids)

# Get cluster assignments for the descriptions
cluster_labels = cluster_assignments.numpy()

# Add cluster labels to the DataFrame
netflix_data['cluster'] = cluster_labels

# Check the results
print(netflix_data[['title', 'description', 'cluster']])


SyntaxError: ignored